<a href="https://colab.research.google.com/github/Syncrei/SmithWatermanCuda/blob/main/SWcu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd drive/MyDrive/SWcu

In [17]:
!nvcc sw.cu -o sw

In [ ]:
%%shell
nvprof ./sw  --db db1.txt --query query1.txt --out out.txt

In [ ]:
%%writefile sw.cu
#include <iostream>
#include <vector>
#include <algorithm>
#include <stdio.h>
#include <fstream>
#include <string>
#include <fstream>
#include <sstream>
#include <iterator>
#include <thrust/device_vector.h>

bool compareSecondElementDescending(const std::pair<int, char>& a, const std::pair<int, char>& b) {
    return a.second > b.second;
}

bool ReadFromFileDB(const std::string& pathToFile, int& index, std::string& seq);
bool ReadDBLimited(const std::string& pathToFile, const long long& maxWeight, const int& querySize, int& maxLen,
    std::vector<int>& indexDb, std::vector<int>& indexChar, std::string& seqDb);
void ReadFileQuery(std::string& a, std::string file_patch);
void WriteFileOut(std::string file_patch, std::vector<std::pair<int, int>> scores, int max_results);

__global__
void CalculateAllDdScoreKernel(int currentStep, char* query, int sizeSeq1, char* dbQuery, int* indexChar, int* scoreArrDb, int* indexStartArr,
    int matchScore, int mismatchScore, int gapScore) {

    int blockId = blockIdx.x;
    int threadId = blockIdx.y * blockDim.x + threadIdx.x;

    int startIdx = (blockId == 0) ? 0 : indexChar[blockId - 1];
    int sizeSeq2 = indexChar[blockId] - startIdx;

    int maxCountDiag = max(sizeSeq1, sizeSeq2)*2 - 1;
    int maxSizeSeq = max(sizeSeq1, sizeSeq2) - 1;

    threadId += max(currentStep - maxSizeSeq, 0);
    int j = threadId + 1;
    int i = currentStep + 2 - j;

    if (i > 0 && j > 0 && i <= sizeSeq1 && j <= sizeSeq2) {

        threadId -= max(currentStep - maxSizeSeq, 0);
        char* seq2 = dbQuery + startIdx;
        int* scoreArrThis = scoreArrDb + indexStartArr[blockId] + (currentStep % 2 == 0 ? 0 : min(sizeSeq1, sizeSeq2));
        int* scoreArrPrev = scoreArrDb + indexStartArr[blockId] + (currentStep % 2 == 0 ? min(sizeSeq1, sizeSeq2) : 0);

        int indexDiag;
        int indexUp;
        int indexLeft;


        if (currentStep > maxCountDiag / 2) {
            if (currentStep > maxCountDiag / 2 + 1)
            {
                indexDiag = threadId + 1;
                indexUp = threadId + 1;
                indexLeft = threadId;
            }
            else
            {
                indexDiag = threadId;
                indexUp = threadId + 1;
                indexLeft = threadId;
            }
        }
        else {
            indexDiag = threadId - 1;
            indexUp = threadId;
            indexLeft = threadId - 1;
        }

        int score_diag = (indexDiag >= 0 && indexDiag < maxCountDiag ? scoreArrThis[indexDiag] : 0) + (query[i - 1] == seq2[j - 1] ? matchScore : mismatchScore);
        int score_up = (indexUp >= 0 && indexUp < maxCountDiag ? scoreArrPrev[indexUp] : 0) + gapScore;
        int score_left = (indexLeft >= 0 && indexLeft < maxCountDiag ? scoreArrPrev[indexLeft] : 0) + gapScore;

        __syncthreads();
        scoreArrThis[threadId] = max(0, max(score_diag, max(score_up, score_left)));
    }
}

__global__
void MaxScoreKernel(int* maxScore, int* scoreArr, int* indexChar, int* indexStartArr, int sizeSeq1, int currentStep, int countSeq)
{
    int threadId = blockIdx.y * blockDim.x + threadIdx.x;
    int startIdx = (threadId == 0) ? 0 : indexChar[threadId - 1];
    int sizeSeq2 = indexChar[threadId] - startIdx;

    if (threadId < countSeq) {
        int* scoreArrThis = scoreArr + indexStartArr[threadId] + (currentStep % 2 == 0 ? 0 : min(sizeSeq1, sizeSeq2));
        for (int i = 0; i < min(sizeSeq2, sizeSeq1); i++) maxScore[threadId] = max(maxScore[threadId], scoreArrThis[i]);
    }
}

void CalculateAllDdScore(std::string db_patch, std::string query_patch, std::string out_patch, int max_results) {
    int matchScore = 1;
    int mismatchScore = -1;
    int gapScore = -2;

    std::string query;
    ReadFileQuery(query, query_patch);

    char* d_query;

    cudaMalloc(&d_query, query.size() * sizeof(char));
    cudaMemcpy(d_query, query.c_str(), query.size() * sizeof(char), cudaMemcpyHostToDevice);

    cudaDeviceProp deviceProp;
    size_t freeMemory, totalMemory;
    cudaGetDeviceProperties(&deviceProp, 0);
    cudaMemGetInfo(&freeMemory, &totalMemory);

    std::vector<int> score;
    std::vector<int> indexes;

    std::vector<int> indexDb;
    std::vector<int> indexChar;
    std::string seqDb;
    int maxLen = 0;

    while (ReadDBLimited(db_patch, (long long)freeMemory * 0.6, query.size(), maxLen, indexDb, indexChar, seqDb))
    {

        int countSeq = indexDb.size();
        int ScoreArrSize = (indexChar[0] - 1) * 2;
        std::vector<int> indexStartArr;
        indexStartArr.push_back(0);
        for (int i = 1; i < countSeq; i++)
        {
            indexStartArr.push_back(ScoreArrSize);
            ScoreArrSize += 2 * std::min(indexChar[i] - indexChar[i - 1], (int)query.size());
        }

        char* d_seqDb;
        int* d_indexChar;
        int* d_indexStartArr;
        int* d_scoreArr;

        cudaMalloc(&d_seqDb, seqDb.size() * sizeof(char));
        cudaMalloc(&d_indexChar, countSeq * sizeof(int));
        cudaMalloc(&d_indexStartArr, countSeq * sizeof(int));
        cudaMalloc(&d_scoreArr, ScoreArrSize * sizeof(int));

        cudaMemcpy(d_seqDb, seqDb.data(), seqDb.size(), cudaMemcpyHostToDevice);
        cudaMemcpy(d_indexChar, indexChar.data(), indexChar.size() * sizeof(int), cudaMemcpyHostToDevice);
        cudaMemcpy(d_indexStartArr, indexStartArr.data(), indexStartArr.size() * sizeof(int), cudaMemcpyHostToDevice);
        cudaMemset(d_scoreArr, 0, ScoreArrSize * sizeof(int));

        int* d_scoreMax;
        cudaMalloc(&d_scoreMax, countSeq * sizeof(int));
        cudaMemset(d_scoreMax, 0, countSeq * sizeof(int));

        int threadsPerBlock = min(1024, min(maxLen, (int)query.size()));
        dim3 numBlocks(countSeq, (min(maxLen, (int)query.size()) + threadsPerBlock - 1) / threadsPerBlock);

        int threadsPerBlock1 = min(1024, countSeq);
        int numBlocks1 = (countSeq + threadsPerBlock - 1) / threadsPerBlock;
        for (int i = 0; i < query.size() + maxLen - 1; i++) {
            CalculateAllDdScoreKernel << <numBlocks, threadsPerBlock >> > (i, d_query, query.size(), d_seqDb, d_indexChar, d_scoreArr, d_indexStartArr,
                matchScore, mismatchScore, gapScore);

            MaxScoreKernel << <numBlocks1, threadsPerBlock1 >> > (d_scoreMax, d_scoreArr, d_indexChar, d_indexStartArr, query.size(), i, countSeq);
            cudaDeviceSynchronize();
        }

        int size = countSeq * sizeof(int);
        int* h_data = (int*)malloc(size);
        cudaMemcpy(h_data, d_scoreMax, size, cudaMemcpyDeviceToHost);
        std::vector<int> vec(h_data, h_data + size / sizeof(int));

        score.insert(score.end(), vec.begin(), vec.end());
        indexes.insert(indexes.end(), indexDb.begin(), indexDb.end());


        indexDb.clear();
        indexChar.clear();
        seqDb.clear();
        maxLen = query.size();

        cudaFree(d_seqDb);
        cudaFree(d_indexChar);
        cudaFree(d_indexStartArr);
        cudaFree(d_scoreArr);
        cudaFree(d_scoreMax);

        delete[] h_data;
    }

    cudaFree(d_query);

    std::vector<std::pair<int, int>> scorAndIndex;
    for (int i = 0; i < indexes.size(); i++) {
        scorAndIndex.push_back(std::make_pair(indexes[i], score[i]));
    }

    std::sort(scorAndIndex.begin(), scorAndIndex.end(), compareSecondElementDescending);

    WriteFileOut(out_patch, scorAndIndex, max_results);

}


int32_t main(int argc, char* argv[]) {


    std::string db_name;
    std::string query_name;
    std::string out_name;
    int max_results = 250;

    for (int i = 1; i < argc; i++) {
        if (std::strcmp(argv[i], "--db") == 0 && i + 1 < argc) {
            db_name = argv[++i];
        }
        else if (std::strcmp(argv[i], "--query") == 0 && i + 1 < argc) {
            query_name = argv[++i];
        }
        else if (std::strcmp(argv[i], "--out") == 0 && i + 1 < argc) {
            out_name = argv[++i];
        }
        else if (std::strcmp(argv[i], "--max_results") == 0 && i + 1 < argc) {
            max_results = std::stoi(argv[++i]);
        }
    }

    if (db_name.empty() || query_name.empty() || out_name.empty()) {
        std::cerr << "Error: Missing parameters! You need to provide --db, --query, and --out.\n";
        return 1;
    }

    CalculateAllDdScore(db_name, query_name, out_name, max_results);
    return 0;
}

void WriteFileOut(std::string file_patch, std::vector<std::pair<int, int>> scores, int max_results) {

    std::ofstream out;
    out.open(file_patch);
    if (out.is_open()) {
        int i = 1;
        for (std::pair<int, int> score : scores) {
            out << score.first << ", " << score.second << "\n";
            if (i++ >= max_results) break;
        }
    }
    out.close();
}

void ReadFileQuery(std::string& a, std::string file_patch) {
    std::ifstream in(file_patch);
    if (in.is_open()) {
        std::getline(in, a);
    }
    in.close();
}

bool ReadFromFileDB(const std::string& pathToFile, int& index, std::string& seq) {
    static std::ifstream file(pathToFile);
    static std::string line;

    if (std::getline(file, line)) {
        std::stringstream ss(line);
        std::string temp;
        if (std::getline(ss, temp, '|')) {
            index = std::stoi(temp);
            std::getline(ss, seq);
            return true;
        }
    }

    file.close();
    return false;
}

bool ReadDBLimited(const std::string& pathToFile, const long long& maxWeight, const int& querySize, int& maxLen,
    std::vector<int>& indexDb, std::vector<int>& indexChar, std::string& seqDb) {
    static std::string lineS;
    static int indexS;

    long long weight = 0;
    if (lineS.size() != 0)
    {
        seqDb += lineS;
        indexDb.push_back(indexS);
        indexChar.push_back(lineS.size());
        weight += lineS.size() * sizeof(char) + std::min((int)lineS.size(), querySize) * sizeof(int) * 2 + sizeof(int) * 3;
        maxLen = std::max(maxLen, (int)lineS.size());
    }

    lineS = "";
    int index;
    std::string seq;

    long long tWeight = 0;
    while (ReadFromFileDB(pathToFile, index, seq))
    {
        tWeight += seq.size() * sizeof(char) + std::min((int)seq.size(), querySize) * sizeof(int) * 2 + sizeof(int) * 3;
        if (tWeight + weight < maxWeight) {
            seqDb += seq;
            indexDb.push_back(index);
            indexChar.push_back(seqDb.size());
            weight += tWeight;
            maxLen = std::max(maxLen, (int)seq.size());
        }
        else {
            lineS = seq;
            indexS = index;
            return true;
        }
    }
    return seqDb.size() == 0 ? false : true;
}

Writing sw.cu
